# Выпускной проект. SQL-запросы к базе данных сервиса по продаже книг.

**Описание проекта**

Компания решила купить крупный сервис для чтения книг по подписке. 

**Основная цель исследования:**

помочь сформулировать ценностное предложение для нового продукта, путем анализа имеющейся базы данных.

Цель подразделяется на задачи, которые необходимо выполнить:

- Вывести по 5 строк каждой таблицы;
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключитим из анализа брошюры;
- Определиим автора с самой высокой средней оценкой книг — необходимо брать в учет только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Описание данных**

Таблица `books` cодержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` cодержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` cодержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings` содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` содержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
#Дополним коллекцию библиотек
#!python -m pip install psycopg2-binary

In [2]:
#Доступ к базе данных

import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#напишем функцию для вывода запросов
def sql(query):
    return pd.io.sql.read_sql(sql=text(query), con = engine)

## Задача 1. *Вывести по 5 строк каждой таблицы*

In [3]:
#Напишем фукнция для автоматизации вывода
def sql_1(source):
    query = '''SELECT * FROM {} LIMIT 5'''.format(source)
    return pd.io.sql.read_sql(sql=text(query), con = engine)

#Выведем все 5 книг
sources = ['books','authors','publishers','ratings','reviews']
for x in sources:
    print('Рассмотрим таблицу {}'.format(x))
    display(sql_1(x))
    print('#' * 150)
    

Рассмотрим таблицу books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


######################################################################################################################################################
Рассмотрим таблицу authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


######################################################################################################################################################
Рассмотрим таблицу publishers


...

######################################################################################################################################################
Рассмотрим таблицу ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


######################################################################################################################################################
Рассмотрим таблицу reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


######################################################################################################################################################


## Задача 2. *Посчитать, сколько книг вышло после 1 января 2000 года*

In [4]:
query = '''SELECT COUNT(*)
           FROM books
           WHERE publication_date > '2000-01-01' '''
sql(query)

,count
0,819


## Задача 3. *Для каждой книги посчитать количество обзоров и среднюю оценку*

In [11]:
query = '''SELECT re.book_id,
                  COUNT(review_id),
                  AVG(rating)
           FROM ratings ra
                           JOIN reviews re 
                                           on ra.book_id = re.book_id
           GROUP BY re.book_id'''

sql(query)

,book_id,count,avg
0,790.0,4,3.500000
1,828.0,4,3.000000
2,NaN,0,3.923077
3,652.0,4,4.500000
4,273.0,4,4.500000
...,...,...,...
990,449.0,9,4.333333
991,520.0,9,4.000000
992,64.0,52,4.230769
993,55.0,4,5.000000


In [18]:
query = ''' WITH 
           a AS
           (SELECT COUNT(rating_id) AS rating,
                   book_id
           FROM ratings 
           GROUP BY book_id),
           
           b AS 
           (SELECT COUNT(review_id) AS review,
                   book_id
            FROM reviews
            GROUP BY book_id)
            
SELECT 
       rating,
       review
FROM
     a LEFT JOIN b ON a.book_id = b.book_id
           '''

sql(query)

,rating,review
0,3,2.0
1,2,1.0
2,3,3.0
3,2,2.0
4,6,4.0
...,...,...
995,3,3.0
996,5,3.0
997,5,4.0
998,2,2.0


## Задача 4. *Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так исключитим из анализа брошюры*

In [6]:
query = '''SELECT COUNT(book_id),
                  p.publisher
           FROM publishers p 
                            JOIN books b 
                                        ON p.publisher_id = b.publisher_id
           WHERE num_pages > 50
           GROUP BY p.publisher 
           ORDER BY 1 DESC
           LIMIT 1
           '''
sql(query)

,count,publisher
0,42,Penguin Books


## Задача 5. Определим автора с самой высокой средней оценкой книг — необходимо брать в учет только книги с 50 и более оценками

In [19]:
query = '''                        SELECT
                           authors.author_id AS id,
                           authors.author AS author,
                           AVG(ratings.rating) AS avg_rating,
                           COUNT(ratings.rating) AS count_rating
                        FROM
                           authors
                        JOIN books ON books.author_id = authors.author_id
                        JOIN ratings ON ratings.book_id = books.book_id
                        GROUP BY
                           id,
                           author
                        HAVING 
                           COUNT(ratings.rating) >= 50 
                           
                        ORDER BY
                           count_rating DESC
                        LIMIT 1
                        ;
           '''
sql(query)

,id,author,avg_rating,count_rating
0,236,J.K. Rowling/Mary GrandPré,4.288462,312


## Задача 6. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [8]:
query = '''SELECT username,
            COUNT(username)
            
           FROM ratings
           
           GROUP BY 1
           HAVING COUNT(username) > 50
           '''
sql(query)

,username,count
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


In [9]:
query = '''SELECT 
                 AVG(counta)
           FROM  (SELECT r.username AS username,
                 COUNT(review_id) AS counta
           FROM reviews r 
                          RIGHT JOIN (SELECT username,
                                             COUNT(username)
                                      FROM ratings
                                      GROUP BY 1
                                      HAVING COUNT(username) > 50) p ON r.username = p.username

           GROUP BY 1) AS t
           '''
sql(query)

,avg
0,24.333333


**Вывод:**

- 819 книг вышло после 1 января 2000 года.
- Посчитали количество обзоров и среднюю оценку
- Издательство - Penguin Books выпускает большее количество книг толще 50 страниц
- Автор с самой высокой средней оценкой - J.K. Rowling/Mary GrandPré (4.41)
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок ровняется 24.3